In [1]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to C:\Users\Ananya
[nltk_data]     saha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import os

file_path = r'C:\Users\Ananya saha\Downloads\Projects\Movie_Recommendation_system\netflix_titles.csv'
print(os.access(file_path, os.R_OK))  # Checks if the file is readable

True


In [8]:
data = pd.read_csv(r'C:\Users\Ananya saha\Downloads\Projects\Movie_Recommendation_system\netflix_titles.csv\netflix_titles.csv')

In [9]:
data.head(6)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...


In [10]:
data.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [11]:
data = data.dropna(subset=['cast', 'country', 'rating'])

In [13]:
movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head(6)

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,PG-13,Comedies
5,Dark Skies,Scott Stewart,"Keri Russell, Josh Hamilton, J.K. Simmons, Dak...",United States,PG-13,"Horror Movies, Sci-Fi & Fantasy"


In [14]:
tv = data[data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV ..."


In [15]:
actors = []

# Split the 'cast' column by commas and store the result in the 'actors' list
for i in movies['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)
    
flat_list = []
# Flatten the list of lists into a single list of actors
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
# Create a sorted list of unique actors
actors_list = sorted(set(flat_list))

# Initialize a binary matrix for actors with zeros
binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]

# Populate the binary matrix with 1s where the actor is present in the movie's cast
for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_actors = pd.DataFrame(binary_actors).transpose()
        
directors = []

# Split the 'director' column by commas and store the result in the 'directors' list
for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
# Flatten the list of lists into a single list of directors
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
# Create a sorted list of unique directors
directors_list = sorted(set(flat_list2))

# Initialize a binary matrix for directors with zeros
binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

# Populate the binary matrix with 1s where the director is present in the movie's director list
for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_directors = pd.DataFrame(binary_directors).transpose()
        
countries = []

# Split the 'country' column by commas and store the result in the 'countries' list
for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
# Flatten the list of lists into a single list of countries
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
# Create a sorted list of unique countries
countries_list = sorted(set(flat_list3))

# Initialize a binary matrix for countries with zeros
binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

# Populate the binary matrix with 1s where the country is present in the movie's country list
for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

# Split the 'listed_in' column by commas and store the result in the 'genres' list
for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
# Flatten the list of lists into a single list of genres
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
# Create a sorted list of unique genres
genres_list = sorted(set(flat_list4))

# Initialize a binary matrix for genres with zeros
binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

# Populate the binary matrix with 1s where the genre is present in the movie's genre list
for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

# Collect all ratings from the 'rating' column
for i in movies['rating']:
    ratings.append(i)

# Create a sorted list of unique ratings
ratings_list = sorted(set(ratings))

# Initialize a binary matrix for ratings with zeros
binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

# Populate the binary matrix with 1s where the rating is present in the movie's rating list
for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [16]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1,ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,...,29105,29106,29107,29108,29109,29110,29111,29112,29113,29114
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
actors2 = []

# Split the 'cast' column by commas and store the result in the 'actors2' list
for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)
    
flat_list5 = []
# Flatten the list of lists into a single list of actors
for sublist in actors2:
    for item in sublist:
        flat_list5.append(item)
        
# Create a sorted list of unique actors
actors_list2 = sorted(set(flat_list5))

# Initialize a binary matrix for actors with zeros
binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

# Populate the binary matrix with 1s where the actor is present in the TV show's cast
for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_actors2 = pd.DataFrame(binary_actors2).transpose()
        

countries2 = []

# Split the 'country' column by commas and store the result in the 'countries2' list
for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
# Flatten the list of lists into a single list of countries
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)
        
# Create a sorted list of unique countries
countries_list2 = sorted(set(flat_list6))

# Initialize a binary matrix for countries with zeros
binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

# Populate the binary matrix with 1s where the country is present in the TV show's country list
for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

# Split the 'listed_in' column by commas and store the result in the 'genres2' list
for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
# Flatten the list of lists into a single list of genres
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
# Create a sorted list of unique genres
genres_list2 = sorted(set(flat_list7))

# Initialize a binary matrix for genres with zeros
binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

# Populate the binary matrix with 1s where the genre is present in the TV show's genre list
for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

# Collect all ratings from the 'rating' column
for i in tv['rating']:
    ratings2.append(i)

# Create a sorted list of unique ratings
ratings_list2 = sorted(set(ratings2))

# Initialize a binary matrix for ratings with zeros
binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

# Populate the binary matrix with 1s where the rating is present in the TV show's rating list
for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

In [18]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
binary2

,0,1,2,3,4,5,6,7,8,9,...,13561,13562,13563,13564,13565,13566,13567,13568,13569,13570
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
def recommender(search):
    cs_list = []  # List to store cosine similarity scores
    binary_list = []  # List to store binary features of the searched title
    
    # Check if the search title is in the movies dataset
    if search in movies['title'].values:
        # Get the index of the searched title
        idx = movies[movies['title'] == search].index.item()
        
        # Get the binary features of the searched title
        for i in binary.iloc[idx]:
            binary_list.append(i)
        
        # Convert the binary features to a 1D array
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        
        # Calculate cosine similarity with all other movies
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        # Create a copy of the movies dataframe and add the cosine similarity scores
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        
        # Sort the movies by cosine similarity in descending order
        results = movies_copy.sort_values('cos_sim', ascending=False)
        
        # Exclude the searched title from the results
        results = results[results['title'] != search]
        
        # Get the top 5 results
        top_results = results.head(5)
        return top_results
    
    # Check if the search title is in the TV shows dataset
    elif search in tv['title'].values:
        # Get the index of the searched title
        idx = tv[tv['title'] == search].index.item()
        
        # Get the binary features of the searched title
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        
        # Convert the binary features to a 1D array
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        
        # Calculate cosine similarity with all other TV shows
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        # Create a copy of the TV shows dataframe and add the cosine similarity scores
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        
        # Sort the TV shows by cosine similarity in descending order
        results = tv_copy.sort_values('cos_sim', ascending=False)
        
        # Exclude the searched title from the results
        results = results[results['title'] != search]
        
        # Get the top 5 results
        top_results = results.head(5)
        return top_results
    
    # If the title is not found in either dataset, return an error message
    else:
        return "Title not in dataset. Please check spelling."

In [20]:
recommender('The Conjuring')

,title,director,cast,country,rating,listed_in,cos_sim
507,Insidious,James Wan,"Patrick Wilson, Rose Byrne, Lin Shaye, Ty Simp...","United States, Canada",PG-13,"Horror Movies, Thrillers",0.424437
3223,Creep,Patrick Brice,"Mark Duplass, Patrick Brice",United States,R,"Horror Movies, Independent Movies, Thrillers",0.416667
603,The Conjuring 2,James Wan,"Patrick Wilson, Vera Farmiga, Madison Wolfe, F...","Canada, United States, United Kingdom",R,Horror Movies,0.412479
1832,In the Tall Grass,Vincenzo Natali,"Patrick Wilson, Laysla De Oliveira, Avery Whit...","Canada, United States",TV-MA,"Horror Movies, Thrillers",0.400892
3728,Desolation,Sam Patton,"Jaimi Paige, Alyshia Ochse, Toby Nichols, Clau...",United States,TV-MA,"Horror Movies, Thrillers",0.395285


In [21]:
recommender('Charlie and the Chocolate Factory')

,title,director,cast,country,rating,listed_in,cos_sim
3835,Figaro Pho,Luke Jurevicius,Luke Jurevicius,United Kingdom,TV-Y7,"Children & Family Movies, Comedies",0.394055
4506,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.390095
4802,Sweeney Todd: The Demon Barber of Fleet Street,Tim Burton,"Johnny Depp, Helena Bonham Carter, Alan Rickma...","United States, United Kingdom",R,"Dramas, Horror Movies, Music & Musicals",0.382692
2606,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.361158
1547,A Shaun the Sheep Movie: Farmageddon,"Richard Phelan, Will Becher","Justin Fletcher, John Sparkes, Amalia Vitale, ...","United Kingdom, France, Belgium, Ireland, Unit...",G,"Children & Family Movies, Comedies",0.354005


In [ ]:
recommender("Dr. Seuss' The Cat in the Hat")

In [ ]:
recommender('Hook')

In [ ]:
recommender('After Life')

In [ ]:
recommender('Anne with an E')

In [ ]:
recommender('Friends')

In [ ]:
recommender('13 Reasons Why')

In [ ]:
recommender('Breaking Bad')

In [ ]:
recommender('Stranger Things')

In [ ]:
movies_des = data[data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
movies_des.head()

In [ ]:
tv_des = data[data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
tv_des.head()

In [ ]:
filtered_movies = []
movies_words = []

# Tokenize the descriptions and remove stopwords
for text in movies_des['description']:
    # Tokenize the text into words
    text_tokens = word_tokenize(text)
    
    # Remove stopwords and convert words to lowercase
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    
    # Append the list of tokens without stopwords to movies_words
    movies_words.append(tokens_without_sw)
    
    # Join the tokens back into a single string
    filtered = (" ").join(tokens_without_sw)
    
    # Append the filtered description to filtered_movies
    filtered_movies.append(filtered)

# Flatten the list of lists into a single list of words
movies_words = [val for sublist in movies_words for val in sublist]

# Create a sorted list of unique words
movies_words = sorted(set(movies_words))

# Add the filtered descriptions as a new column in the dataframe
movies_des['description_filtered'] = filtered_movies

# Display the first few rows of the dataframe
movies_des.head()

In [ ]:
filtered_tv = []
tv_words = []

# Tokenize the descriptions and remove stopwords
for text in tv_des['description']:
    # Tokenize the text into words
    text_tokens = word_tokenize(text)
    
    # Remove stopwords and convert words to lowercase
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    
    # Append the list of tokens without stopwords to tv_words
    tv_words.append(tokens_without_sw)
    
    # Join the tokens back into a single string
    filtered = (" ").join(tokens_without_sw)
    
    # Append the filtered description to filtered_tv
    filtered_tv.append(filtered)

# Flatten the list of lists into a single list of words
tv_words = [val for sublist in tv_words for val in sublist]

# Create a sorted list of unique words
tv_words = sorted(set(tv_words))

# Add the filtered descriptions as a new column in the dataframe
tv_des['description_filtered'] = filtered_tv

# Display the first few rows of the dataframe
tv_des.head()

In [ ]:
# Initialize a binary matrix for movie descriptions with zeros
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

# Populate the binary matrix based on the presence of words in the filtered descriptions
for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

In [ ]:
# Initialize a binary matrix for TV show descriptions with zeros
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

# Populate the binary matrix based on the presence of words in the filtered descriptions
for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k += 1
        
# Convert the binary matrix to a DataFrame and transpose it
tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

def recommender2(search):
    cs_list = []  # List to store cosine similarity scores
    binary_list = []  # List to store binary features of the searched title
    
    # Check if the search title is in the movies dataset
    if search in movies_des['title'].values:
        # Get the index of the searched title
        idx = movies_des[movies_des['title'] == search].index.item()
        
        # Get the binary features of the searched title
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        
        # Convert the binary features to a 1D array
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        
        # Calculate cosine similarity with all other movies
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        # Create a copy of the movies dataframe and add the cosine similarity scores
        movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        
        # Sort the movies by cosine similarity in descending order
        results = movies_copy.sort_values('cos_sim', ascending=False)
        
        # Exclude the searched title from the results
        results = results[results['title'] != search]
        
        # Get the top 5 results
        top_results = results.head(5)
        return top_results
    
    # Check if the search title is in the TV shows dataset
    elif search in tv_des['title'].values:
        # Get the index of the searched title
        idx = tv_des[tv_des['title'] == search].index.item()
        
        # Get the binary features of the searched title
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        
        # Convert the binary features to a 1D array
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        
        # Calculate cosine similarity with all other TV shows
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        
        # Create a copy of the TV shows dataframe and add the cosine similarity scores
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        
        # Sort the TV shows by cosine similarity in descending order
        results = tv_copy.sort_values('cos_sim', ascending=False)
        
        # Exclude the searched title from the results
        results = results[results['title'] != search]
        
        # Get the top 5 results
        top_results = results.head(5)
        return top_results
    
    # If the title is not found in either dataset, return an error message
    else:
        return "Title not in dataset. Please check spelling."

In [ ]:
pd.options.display.max_colwidth = 300
recommender2('The Conjuring')

In [ ]:
recommender2("Child's Play")

In [ ]:
recommender2('Wild Child')

In [ ]:
recommender2("Dr. Seuss' The Cat in the Hat")

In [ ]:
recommender2("Holidate")

In [ ]:
recommender2('After Life')

In [ ]:
recommender2('Gilmore Girls')

In [ ]:
recommender2('Love on the Spectrum')

In [ ]:
recommender2('13 Reasons Why')

In [ ]:
recommender2('Breaking Bad')